# PandasDataFrameOutputParser

Hướng dẫn này xử lý cách kiểm soát đầu ra của LLM dưới định dạng `pd.DataFrame`.

Pandas là một package, hữu ích trong định dạng bảng (ví dụ: dữ liệu dạng bảng - tabular data), được sử dụng rộng rãi bởi các nhà khoa học dữ liệu. Nó sẽ giúp bạn khám phá, làm sạch và xử lý dữ liệu.

Để tìm hiểu thêm về `pd.DataFrame` và các khả năng của nó, hãy truy cập một trong những hướng dẫn chính thức của Pandas, [10 minutes to pandas](https://pandas.pydata.org/docs/user_guide/10min.html).

`PandasDataFrameOutputParser` phân tích đầu ra sử dụng định dạng Pandas DataFrame, theo [tài liệu API chính thức](https://python.langchain.com/api_reference/langchain/output_parsers/langchain.output_parsers.pandas_dataframe.PandasDataFrameOutputParser.html)
- Điều này có thể được sử dụng cả như việc tạo đầu ra có cấu trúc cho truy vấn LLM dưới dạng chuỗi và như việc tạo đầu vào có cấu trúc cho truy vấn LLM
    - Là đầu ra, bạn có thể xem thêm cách sử dụng pydantic trong [01-PydanticOutputParser](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/main/03-OutputParser/01-PydanticOutputParser.ipynb) và các phần khác trong hướng dẫn học tập này.
    - Là đầu vào, tập dữ liệu `pd.DataFrame` có thể được sử dụng để LLM tương tác với dữ liệu.

**Mục tiêu của hướng dẫn này**

- Biết nơi `PandasDataFrameOutputParser` được sử dụng để tương tác với `pd.DataFrame`

**Thu thập dữ liệu (Data Acqusition)**
- Truy cập vào [figshare.com](https://figshare.com) nơi bạn có thể khám phá đầu ra của nghiên cứu học thuật từ csv đến pdf.


In [1]:
# Setup environment
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv(override=True, dotenv_path="../.env")

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.0)

## Without model use
- The `format_parser_output` function is used to convert parser output to dictionary format and format the output.
- When input is `pd.DataFrame` or file. 

In [2]:
import pprint
from typing import Any, Dict
import os
import pandas as pd
from langchain.output_parsers import PandasDataFrameOutputParser
from langchain_core.prompts import PromptTemplate

# Purpose: Only to print. It is a helper function.
def format_parser_output(parser_output: Dict[str, Any]) -> None:
    length_val = len(list(parser_output.values()))
    if length_val == 1 and isinstance(parser_output.values, (int, float)):
        return {"result": parser_output}
        # Iterate around the "keys" of parser output (dict)
    else:
        for key in parser_output.keys():
            parser_output[key] = parser_output[key].to_dict()
            # Pretty Print
            return pprint.PrettyPrinter(width=4, compact=True).pprint(parser_output)

In [3]:
# Define dataframe
df = pd.read_csv("./data/Air-Traffic-Passenger-Statistics-csv.csv")
df.head()

,Activity Period,Activity Period Start Date,Operating Airline,Operating Airline IATA Code,Published Airline,Published Airline IATA Code,GEO Summary,GEO Region,Activity Type Code,Price Category Code,Terminal,Boarding Area,Passenger Count,data_as_of,data_loaded_at
0,199907,1999/07/01,ATA Airlines,TZ,ATA Airlines,TZ,Domestic,US,Deplaned,Low Fare,Terminal 1,B,31432,2023/11/20 07:01:34 AM,2023/11/20 07:02:25 AM
1,199907,1999/07/01,ATA Airlines,TZ,ATA Airlines,TZ,Domestic,US,Enplaned,Low Fare,Terminal 1,B,31353,2023/11/20 07:01:34 AM,2023/11/20 07:02:25 AM
2,199907,1999/07/01,ATA Airlines,TZ,ATA Airlines,TZ,Domestic,US,Thru / Transit,Low Fare,Terminal 1,B,2518,2023/11/20 07:01:34 AM,2023/11/20 07:02:25 AM
3,199907,1999/07/01,Aeroflot Russian International Airlines,NaN,Aeroflot Russian International Airlines,NaN,International,Europe,Deplaned,Other,Terminal 2,D,1324,2023/11/20 07:01:34 AM,2023/11/20 07:02:25 AM
4,199907,1999/07/01,Aeroflot Russian International Airlines,NaN,Aeroflot Russian International Airlines,NaN,International,Europe,Enplaned,Other,Terminal 2,D,1198,2023/11/20 07:01:34 AM,2023/11/20 07:02:25 AM


In [4]:
df.columns

Index(['Activity Period', 'Activity Period Start Date', 'Operating Airline',
       'Operating Airline IATA Code', 'Published Airline',
       'Published Airline IATA Code', 'GEO Summary', 'GEO Region',
       'Activity Type Code', 'Price Category Code', 'Terminal',
       'Boarding Area', 'Passenger Count', 'data_as_of', 'data_loaded_at'],
      dtype='object')

In [5]:
df.columns = df.columns.str.strip().str.replace('"', "").str.replace("'", "")

In [6]:
# Set up a parser and inject instructions into the prompt template.
parser = PandasDataFrameOutputParser(dataframe=df)
# Print the parser's instructions.
print(parser.get_format_instructions())

The output should be formatted as a string as the operation, followed by a colon, followed by the column or row to be queried on, followed by optional array parameters.
1. The column names are limited to the possible columns below.
2. Arrays must either be a comma-separated list of numbers formatted as [1,3,5], or it must be in range of numbers formatted as [0..4].
3. Remember that arrays are optional and not necessarily required.
4. If the column is not in the possible columns or the operation is not a valid Pandas DataFrame operation, return why it is invalid as a sentence starting with either "Invalid column" or "Invalid operation".

As an example, for the formats:
1. String "column:num_legs" is a well-formatted instance which gets the column num_legs, where num_legs is a possible column.
2. String "row:1" is a well-formatted instance which gets row 1.
3. String "column:num_legs[1,2]" is a well-formatted instance which gets the column num_legs for rows 1 and 2, where num_legs is a p

In [7]:
print(df["GEO Region"].head())

0        US
1        US
2        US
3    Europe
4    Europe
Name: GEO Region, dtype: object


In [8]:
df.columns[7]

'GEO Region'

In [9]:
# Example of working with columns.

df_query = "column:GEO Region[0..4]"  # Replace with your intended query
output = parser.parse(df_query)
print(output)
print("----")
print("----")

{'GEO Region': 0        US
1        US
2        US
3    Europe
4    Europe
Name: GEO Region, dtype: object}
----
----


In [10]:
# Example of retrieving first row
df_query = "row:1"

# Row lookup
output = parser.parse(df_query)
format_parser_output(output)

{'1': {'Activity Period': 199907,
       'Activity Period Start Date': '1999/07/01',
       'Activity Type Code': 'Enplaned',
       'Boarding Area': 'B',
       'GEO Region': 'US',
       'GEO Summary': 'Domestic',
       'Operating Airline': 'ATA '
                            'Airlines',
       'Operating Airline IATA Code': 'TZ',
       'Passenger Count': 31353,
       'Price Category Code': 'Low '
                              'Fare',
       'Published Airline': 'ATA '
                            'Airlines',
       'Published Airline IATA Code': 'TZ',
       'Terminal': 'Terminal '
                   '1',
       'data_as_of': '2023/11/20 '
                     '07:01:34 '
                     'AM',
       'data_loaded_at': '2023/11/20 '
                         '07:02:25 '
                         'AM'}}


In [11]:
df["Passenger Count"].head(5)

0    31432
1    31353
2     2518
3     1324
4     1198
Name: Passenger Count, dtype: int64

In [12]:
# Average passenger counts from 5 first rows
df["Passenger Count"].head(5).mean()

13565.0

In [13]:
# Querying the same operation as above
df_query = "mean:Passenger Count[0..4]"

output = parser.parse(df_query)
print(output)

{'mean': 13565.0}


In [14]:
df.head()

,Activity Period,Activity Period Start Date,Operating Airline,Operating Airline IATA Code,Published Airline,Published Airline IATA Code,GEO Summary,GEO Region,Activity Type Code,Price Category Code,Terminal,Boarding Area,Passenger Count,data_as_of,data_loaded_at
0,199907,1999/07/01,ATA Airlines,TZ,ATA Airlines,TZ,Domestic,US,Deplaned,Low Fare,Terminal 1,B,31432,2023/11/20 07:01:34 AM,2023/11/20 07:02:25 AM
1,199907,1999/07/01,ATA Airlines,TZ,ATA Airlines,TZ,Domestic,US,Enplaned,Low Fare,Terminal 1,B,31353,2023/11/20 07:01:34 AM,2023/11/20 07:02:25 AM
2,199907,1999/07/01,ATA Airlines,TZ,ATA Airlines,TZ,Domestic,US,Thru / Transit,Low Fare,Terminal 1,B,2518,2023/11/20 07:01:34 AM,2023/11/20 07:02:25 AM
3,199907,1999/07/01,Aeroflot Russian International Airlines,NaN,Aeroflot Russian International Airlines,NaN,International,Europe,Deplaned,Other,Terminal 2,D,1324,2023/11/20 07:01:34 AM,2023/11/20 07:02:25 AM
4,199907,1999/07/01,Aeroflot Russian International Airlines,NaN,Aeroflot Russian International Airlines,NaN,International,Europe,Enplaned,Other,Terminal 2,D,1198,2023/11/20 07:01:34 AM,2023/11/20 07:02:25 AM


In [15]:
# Wrongly formatted query
df_query = "Calculate value_counts for `GEO` Region."

# Parse out the query
output = parser.parse(df_query)

# Print out result
print(output)

OutputParserException: Request 'Calculate value_counts for `GEO` Region.' is not correctly formatted.                     Please refer to the format instructions.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 

In [16]:
# Well formatted query
df_query = "value_counts:GEO Region"

# Parse out the query
output = parser.parse(df_query)

# Print out result
print(output)

{'value_counts': GEO Region
US                     12475
Asia                    7499
Europe                  5758
Canada                  3216
Mexico                  2651
Australia / Oceania     1712
Central America          809
Middle East              667
South America             91
Name: count, dtype: int64}


In [17]:
# Confirmed with the results
df["Operating Airline"].value_counts()

Operating Airline
United Airlines - Pre 07/01/2013    3670
United Airlines                     3368
SkyWest Airlines                    2028
Alaska Airlines                     1790
Delta Air Lines                      772
                                    ... 
Pacific Aviation                       2
Casino Express                         1
Boeing Company                         1
Samsic Airport America, LLC            1
Harmony Airways                        1
Name: count, Length: 134, dtype: int64

## With model

In [20]:
# Query the column; Passenger count
df_query = "Query {Passenger count} column"

# Setting up the PromptTemplate
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],  # Set up input variable. Look above {query}
    partial_variables={
        "format_instructions": parser.get_format_instructions()
    },  # Set up partial variable (how it would like to be displayed)
)

# Set up the chain
chain = prompt | llm

# Execute the chain based on the query
chain_output = chain.invoke({"query": df_query})

# Use parser here
parser_output = parser.parse(chain_output.content[1:-1])

# Print out
format_parser_output(parser_output)

OutputParserException: Unsupported request type 'nvalid column'.                         Please check the format instructions.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 